# Introduction to LangChain 🌐

LangChain is a powerful framework designed to simplify the development of applications that leverage language models. It provides a suite of tools and abstractions to help developers build, deploy, and manage language model-based applications efficiently. Whether you're working on chatbots, text generation, or any other NLP task, LangChain has got you covered! 🚀

## Key Features ✨

- **Modular Design**: Easily integrate with various language models and data sources.
- **Scalability**: Built to handle large-scale applications with ease.
- **Extensibility**: Customize and extend functionalities to suit your specific needs.
- **Community Support**: Active community and extensive documentation to help you get started.


In [ ]:
%pip install -qU langchain-openai==0.2.14
%pip install langchain==0.3.13
%pip install python-dotenv==1.0.1

### Load environments variables

In [1]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

chat_deployment_name = os.getenv("CHAT_DEPLOYMENT_NAME")
api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("OPENAI_BASE_URL")

### Create the client

In [2]:
from langchain_openai import AzureChatOpenAI

client = AzureChatOpenAI(
    azure_endpoint=base_url,
    azure_deployment=chat_deployment_name,
    api_key=api_key,
    api_version="2024-08-01-preview"
)

### Create the first prompt

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

system_message = SystemMessage("""
                  You are an assistant and speak french canadian all the time, you cannot speak english.  
                  You answer with really strong quebecois accent.
                  All your answer always come or are related to the province of Quebec.""")

messages = [
    system_message,
    HumanMessage("Hi where I can find the coolest place to do hiking in Canada?"),
]

client.invoke(messages)


### 📜 Prompt Templates

Right now, we are passing a list of messages directly into the language model. 🤔 But where does this list of messages come from? Usually, it is constructed from a combination of user input and application logic. 🧩

This application logic typically takes the raw user input and transforms it into a list of messages ready to pass to the language model. 🔄 Common transformations include adding a system message or formatting a template with the user input. 📝

In [5]:
from langchain_core.prompts import ChatPromptTemplate

system_message = SystemMessage("""
        You are a food assistant, you receive a city and you provide the most popular food in that city based on user {preference}.
""")

prompt_template = ChatPromptTemplate.from_messages(
    [
        system_message,
        ("user","{city}"),
    ]
)

In [ ]:
# We build the prompt

prompt = prompt_template.invoke({
    "preference": "not healhty and really quebecois",
    "city": "Chicoutimi"
})

prompt.to_messages()

In [ ]:
# Call the model

response = client.invoke(prompt)

print(response.content)

# Introduction to Chains in LangChain ⛓️

Chains in LangChain are a powerful way to link together multiple components to create complex workflows. By connecting various modules such as language models, tools, and APIs, you can build sophisticated applications that leverage the strengths of each component. Chains allow you to define a sequence of operations, making it easier to manage and maintain your code.

With LangChain, you can create both simple and complex chains to handle tasks like data processing, natural language understanding, and more. Whether you're building a chatbot, an automated data analysis tool, or any other application, chains provide a flexible and modular approach to development. 🚀

In [ ]:
from langchain.schema.output_parser import StrOutputParser

template = "Tell me a joke about {topic}."

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful AI bot. Your name is Carl."),
    ("human", template),
])

# Use a string parser and to avoid need to do result.content
#  So this add the prompt --> chat --> end you parse the output
chain = prompt_template | client | StrOutputParser()

result = await chain.ainvoke({"topic": "cat"})

result

In [ ]:
from langchain.schema.runnable import RunnableLambda

prompt_template = ChatPromptTemplate([
    ("system", "You are a comedian who tells jokes about {topic}"),
    ("human", "Tell me {joke_count} jokes."),
])

uppercase_output = RunnableLambda(lambda x: x.upper())
count_words = RunnableLambda(lambda x: f"Words count: {len(x.split())}\n{x}")

chain = prompt_template | client | StrOutputParser() | uppercase_output | count_words

result = await chain.ainvoke({"topic": "cat", "joke_count": 3})

print(result)

In [ ]:
from langchain.schema.runnable import RunnableLambda, RunnableParallel

prompt_template = ChatPromptTemplate([
    ("system", "You are an expert product reviewer"),
    ("human", "List the main features of the product {product_name}.")
])

def analyze_pros(features):
    pros_template = ChatPromptTemplate([
        ("system", "You are an expert product reviewer"),
        ("human", "Given these features: {features}, list the pros of these features.")
    ])
    return pros_template.format_prompt(features=features)

def analyze_cons(features):
    cons_template = ChatPromptTemplate([
        ("system", "You are an expert product reviewer"),
        ("human", "Given these features: {features}, list the cons of these features.")
    ])
    return cons_template.format_prompt(features=features)

def combine_pros_cons(pros, cons):
    return f"Pros:\n {pros}\n\nCons:\n {cons}"

pros_branch = (
    RunnableLambda(lambda x: analyze_pros(x)) | client | StrOutputParser()
)

cons_branch = (
    RunnableLambda(lambda x: analyze_cons(x)) | client | StrOutputParser()
)

chain = (
    prompt_template
    | client
    | StrOutputParser()
    | RunnableParallel(branches={"pros": pros_branch, "cons": cons_branch})
    | RunnableLambda(lambda x: combine_pros_cons(x["branches"]["pros"], x["branches"]["cons"])) 
)

result = await chain.ainvoke({"product_name": "iPhone 13"})

result

### Tool

In [13]:
# Define the function

from pydantic import BaseModel, Field
from langchain_core.tools import tool

@tool
def get_weather(region: str) -> str:
    """Get the regional weather for Middle-Earth from the region passed as a parameter"""
    weather_data = {
        "Shire": "Sunny",
        "Mordor": "Hot and dry",
        "Rivendell": "Mild and rainy",
        "Gondor": "Warm and breezy",
        "Rohan": "Windy",
        "Mirkwood": "Foggy and damp",
        "Isengard": "Stormy",
        "Lothlorien": "Pleasant and cool"
    }
    
   # Normalize the region name to lower case for comparison
    region = region.lower()
    
    for key in weather_data:
        if key.lower() in region:
            return weather_data[key]
    
    return "Unknown region"

In [14]:
tools = [get_weather]

client_with_tools = client.bind_tools(tools)

client_with_tools.invoke("What is the weather in the Mordor")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_i1AxxC4ajVjSjEtnguRxrWtC', 'function': {'arguments': '{"region":"Mordor"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 61, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_82ce25c0d4', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results':

In [15]:
client_with_tools.invoke("What is the weather in the Shire").tool_calls

[{'name': 'get_weather',
  'args': {'region': 'Shire'},
  'id': 'call_WqJ2nO50T4P07RdQndEbTVPv',
  'type': 'tool_call'}]

In [16]:
tools = [get_weather]

client_with_tools = client.bind_tools(tools)

ai_tools = client_with_tools.invoke("What is the weather in the Mordor")

messages = []

for tool_call in ai_tools.tool_calls:
    selected_tool = {"get_weather": get_weather}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[ToolMessage(content='Hot and dry', name='get_weather', tool_call_id='call_oDPh7z1fmaAbio5uYEaKeLGr')]